This script expects two input data sources:

- NLCD data rasters located at `data/raw/nlcd/`
  - These can be downloaded at https://www.mrlc.gov/data
  - In this script I load compressed `*.tif` files created from the raw NLCD `*.img` files
- Shapefiles of all US census tracts located at `data/raw/shapes/`
  - These can be downloaded at https://geodata.socialexplorer.com/ or https://www.census.gov/cgi-bin/geo/shapefiles/index.php


In [1]:
import sys, os, time

import numpy as np
import pandas as pd

import rasterio
import rasterio.mask
import rasterio.io
import rasterio.errors
import fiona
import fiona.transform
import shapely
import shapely.geometry

import collections

## Find the set of census tracts that remain the same over all of our years of data

Here we assume that the shape of census tracts with the same GEOIDs don't change. I'm not sure if this is a safe assumption.

In [2]:
shape_fns = [
    "data/raw/shapes/TRACT_2011_US_SL140_Coast_Clipped.shp",
    "data/raw/shapes/TRACT_2013_US_SL140_Coast_Clipped.shp",
    "data/raw/shapes/TRACT_2016_US_SL140_Coast_Clipped.shp"
]
nlcd_fns = [
    "data/raw/nlcd/nlcd_2011.tif",
    "data/raw/nlcd/nlcd_2013.tif",
    "data/raw/nlcd/nlcd_2016.tif"
]


tracts_per_year = [
    set()
    for fn in shape_fns
]

for i, fn in enumerate(shape_fns):
    with fiona.open(fn, "r") as f:
        for line in f:
            tracts_per_year[i].add(line["properties"]["GEOID"])

In [3]:
for tracts in tracts_per_year:
    print(len(tracts))

73837
72790
73799


In [4]:
common_tracts = tracts_per_year[0]
for tracts in tracts_per_year[1:]:
    common_tracts = common_tracts.intersection(tracts)
print(len(common_tracts))

72749


## Load shapes for the set of census tracts we will use

In [5]:
# sanity check, make sure all of our raster data uses the same coordinate system
with rasterio.open("data/raw/nlcd/nlcd_2016.tif", "r") as f:
    target_crs = f.crs.to_string()
    
for fn in nlcd_fns:
    with rasterio.open(fn, "r") as f:
        assert target_crs == f.crs.to_string()

In [6]:
# extract the census tract shapes from the latest set of definitions and convert them to the coordinate system used by NLCD 
census_tract_geoids = []
census_tract_shapes = []
with fiona.open(shape_fns[-1], "r") as f:
    for line in f:
        geoid = line["properties"]["GEOID"]
        if geoid in common_tracts:
            geom = line["geometry"]
            geom = fiona.transform.transform_geom(f.crs, target_crs, geom)

            census_tract_geoids.append(geoid)
            census_tract_shapes.append(geom)

In [7]:
# write out a new shapefile with the set of shapes that we will be using
os.makedirs("data/processed/", exist_ok=True)
with fiona.open(shape_fns[-1], 'r') as source:
    with fiona.open(
            "data/processed/shapes/output_tracts.shp", "w",
            crs=source.crs,
            driver=source.driver,
            schema=source.schema,
            ) as sink:

        for f in source:
            geoid = f["properties"]["GEOID"]
            if geoid in common_tracts:
                sink.write(f)

## Mask the NLCD data with each shape we care about and count the values within the mask

In [8]:
NLCD_CLASS_VALS = [11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 52, 71, 81, 82, 90, 95]

In [ ]:
%%time

bad_geoids = set() # keep track of all the census tracts that do not have any intersections

for i, fn in enumerate(nlcd_fns):
    print(fn)
    
    counts = np.zeros((len(census_tract_geoids), len(NLCD_CLASS_VALS)), dtype=int)
    
    tic = float(time.time())
    with rasterio.io.MemoryFile(open(fn, "rb")) as memfile: # store the raster data in memory to avoid disk I/O

        for j, (geoid, shape) in enumerate(zip(census_tract_geoids, census_tract_shapes)):
            if j % 5000 == 0 and j != 0:
                print(j, len(census_tract_geoids), time.time()-tic)
                
            with memfile.open() as f:
                try:
                    out_data, out_transform = rasterio.mask.mask(f, [shape], crop=True, all_touched=True)
                    for k, val in enumerate(NLCD_CLASS_VALS):
                        counts[j,k] = (out_data == val).sum()
                    
                    if counts[j].sum() == 0:
                        bad_geoids.add(geoid)
                except (rasterio.errors.WindowError, ValueError):
                    bad_geoids.add(geoid)
                    
    np.save("data/processed/" + os.path.basename(fn)[:-4] + "_per_output_tracts.npy", counts)

data/raw/nlcd/nlcd_2011.tif
5000 72749 886.6057615280151
10000 72749 1387.7164342403412


In [84]:
f = open("data/processed/output_tracts_geoids.txt","w")
for geoid in census_tract_geoids:
    f.write("%s\n" % (geoid))
f.close()

In [ ]:
f = open("data/processed/output_tracts_bad_geoids.txt","w")
for geoid in bad_geoids:
    f.write("%s\n" % (geoid))
f.close()